** 声明 **  
这篇和上一篇关于TensorFlow的基本使用来自公众号[深度学习这件小事](https://mp.weixin.qq.com/s/ubFYvF7YzQ9yQ-F4yEAq4w)  
* 我是因为自己想要总体复习一下Tensorflow,学习Mxnet，想分别使用Tensorflow 和MxNet复现R-CNN 系列论文和SSD,我习惯做事情前先从宏观把握，这里  
感谢该公众号，我基本不需要使用什么的时候，现查Tensorflow官网，毕竟有些慢，还要翻墙

## 命名变量和张量

* 每次调用`tf.get_variable()`时，都需要为变量赋一个新的唯一的名称，图中每个张量也是这样的。可以通过张量/操作和变量的`.name`属性访问该节点的  
的名称。大多情况下，名称会自动创建，也可以通过`name=`属性设置节点名称，列表后缀仍会自动添加

In [2]:
import tensorflow as tf

In [2]:
a = tf.constant(0.)
b = tf.constant(1.)
c = tf.constant(2.,name = "const")
d = tf.constant(3.,name = "const")
print(a.name,b.name,c.name,d.name)

Const:0 Const_1:0 const_2:0 const_3:0


* 我基本上在训练模型时都会给节点命名，方便调试

## 自动命名

tensorflow 提供了`tf.variable_scope`实现自动命名，它通过将图形细分为更小的组块，使图形更加有序，将一段图形创建代码封装在  
`with.tf.variable_scope(scope_name):`中，所有在这个块中的节点都会以`scope_name`作为名称前缀，加入`summary`，可以使用tensorboard可视化

In [3]:
a = tf.constant(0.)
b = tf.constant(1.)
with tf.variable_scope("sum"):
    c = a +b
    d = tf.constant(2.,name = "const")
    coef1 = tf.get_variable("coef",[],initializer = tf.constant_initializer(2.))
    with tf.variable_scope("multi"):
        e = coef1 * d
        coef2 = tf.get_variable("coef",[],initializer = tf.constant_initializer(3.))
        f = tf.constant(4.)
        d = coef2 * f
        
print(a.name,b.name)
print(c.name,d.name)
print(e.name,f.name)
print(coef1.name)
print(coef2.name)

Const_4:0 Const_5:0
sum/add:0 sum/multi/mul_1:0
sum/multi/mul:0 sum/multi/Const:0
sum/coef:0
sum/multi/coef:0


* 上面的命名中，我吗两次使用了`coef`命令，但是由打印结果可以看出，一个是`sum/coef:0`,一个是`sum.multi/coef:0`,前面忘了说，在一个域内创建  
子作用域，会使用/将前缀连到一起

## 保存和加载

* 训练好的神经网络包括两个基本的部分：  
  * 已经学习到的参数  
  * 说明如何利用权重获得结果的网络结构图  
* tensorflow 会将这两部分分开，但它们需要紧密匹配。如果没有图结构进行说明，权重就无用，而带有随机权重的图效果也不好。事实上，即使仅交换  
两个权重矩阵的图效果也不好。事实上，即使仅交换两个权重矩阵也可能完全破坏模型。这通常会让Tensorflow初学者感到挫败。使用预先训练好的模型  
作为神经网络的一个组成部分不失为加速训练的好方法，但是，也可能搞砸一切  

### 保存模型  
当只有单个模型时，TensorFlow用于保存和加载的内置工具使用很方便：只需创建一个`tf.train.Saver()`。类似于`tf.train.Optimizer`,`tf.train.Saver`  
本身并不是一个节点，而是在已有图形上执行有用功能的更高级类别。你可能已经预料到`tf.train`的有用功能了：保存和加载模型  

In [4]:
a = tf.get_variable('a',[])
b = tf.get_variable('b',[])
init = tf.global_variables_initializer()

saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.save(sess,'./tftcp.model')

'./tftcp.model'

In [5]:
?tf.train.Saver()

生成了四个文件在当前目录下![](model.png)

具体内容如下：  
首先：当我们只保存一个模型时，为什么会输出四个文件？ 重建模型所需的信息被分散到他们当中。如果想复制或者备份模型，需要所有四个文件（前缀是  
是文件名）。答案简述：  
* `tftcp.model.data-00000-of-00001`包含模型权重（上述第一个要点），它可能是这四个中最大的  
* `tftcp.model.meta`是模型的网络结构（上述第二个要点）。它包含重建图形所需的所有信息  
* `tftcp.model.index` 是连接前两点的索引结构。用于在数据文件中找到对应节点的参数  
* `checkpoint`实际上不需要重建模型，但如果需要在整个训练过程中保存了多个版本模型，它会跟踪所有内容  
其次，为什么一定要为该示例创建`tf.Session()`和`tf.global_variables_initializer()`呢？  
因为，如果要保存一个模型，我们需要保存相关内容。计算存于图中，但数值存于会话中。`tf.train.Saver()`可以通过指向图的全局指针访问网络结构。但  
当我们保存变量的值（网络权重）时，我们需要访问`tf.Session()`来确定这些值；所以，`sess`作为`save()`的第一个参数传入。此外，尝试保存未初始化  
变量的值总会引发错误。因此，需要一个会话和一个初始化程序（或等价的`tf.assign()`）

### 加载模型  

保存模型的目的就是为了后续加载。重新加载步骤：  
* 第一：重新创建变量，当然，我们希望变量的名称、类型、形状都与保存时一致。  
* 第二：创建与之前一样的`tf.train.Saver()`，并调用`restore()`

In [3]:
a = tf.get_variable('a',[])
b = tf.get_variable('b',[])
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess,'./tftcp.model')
sess.run([a,b])

INFO:tensorflow:Restoring parameters from ./tftcp.model


[1.3019966, -0.097077847]

* 因为，我依然使用的a,b作为变量名称，run时会提示已经存在，可以重启下kernel

* 可以看出`restore`时没有初始化操作，因为`restore`是将保存的值从文件中读到会话变量中。由于会话不再包含任何空值变量，因此，不需要  
    初始化（如果不小心初始化，会适得其反，还原后运行`init`会使随机初始化的值覆盖加载的值）

### 选择变量

当一个`tf.train.Saver()`程序初始化后，它会查看出当前图并获取变量列表；这是`saver`关心的永久存储的变量列表，可以用`._var_list`属性检查

In [4]:
c = tf.get_variable('c',[])
d = tf.get_variable('d',[])
saver = tf.train.Saver()
e = tf.get_variable('e',[])
print(saver._var_list)

[<tf.Variable 'a:0' shape=() dtype=float32_ref>, <tf.Variable 'b:0' shape=() dtype=float32_ref>, <tf.Variable 'c:0' shape=() dtype=float32_ref>, <tf.Variable 'd:0' shape=() dtype=float32_ref>]


我们可以看到列表中有`a`,`b`,`c`,`d`,但是没有`e`，因为创建`saver`时`e`还没有出现，所以，它并没有成为函数的一部分。一般来说，需要在创建  
`saver`之前确保已经创建了所有的变量  
当然，在某些特定情况下，可能只需保存变量的一个子集。当创建`var_list`以期望它跟踪可用变量子集时，`tf.train.Saver()`允许传递`var_list`

In [5]:
f = tf.get_variable('f',[])
g = tf.get_variable('g',[])
h = tf.get_variable('h',[])
saver = tf.train.Saver(var_list = [f,g])
print(saver._var_list)

[<tf.Variable 'f:0' shape=() dtype=float32_ref>, <tf.Variable 'g:0' shape=() dtype=float32_ref>]


### 加载修正模型

上面的例子中涵盖的模型加载方案类似于物理中的真空中无摩擦的完美球体场景，只要你使用自己的代码保存和加载模型，且不擅自更改二者，实现保存和  
加载轻而易举。但是很多情况下，并不会有如此完美的场景。这些情况下，就需要多思考  
通过几个场景说明问题。首先，如果，我们想保存一个完美的模型，

In [2]:
import tensorflow as tf
i = tf.get_variable('i',[])
j = tf.get_variable('j',[])
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.save(sess,'./tfmodify.model')

'./tfmodify.model'

In [1]:
import tensorflow as tf
i = tf.get_variable('i',[])
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.restore(sess,'./tfmodify.model')
sess.run(i)

INFO:tensorflow:Restoring parameters from ./tfmodify.model


-1.0371007

当我们在相反的场景里，就会出现失败的状况：我们希望将一个模型作为大型模型的组件加载

In [1]:
import tensorflow as tf
a = tf.get_variable('a',[])
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.save(sess,'./tftcp.model')

'./tftcp.model'

In [1]:
import tensorflow as tf 
a = tf.get_variable('a',[])
d = tf.get_variable('d',[])
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.restore(sess,'./tftcp.model')

INFO:tensorflow:Restoring parameters from ./tftcp.model


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key d not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op 'save/RestoreV2', defined at:
  File "C:\Users\Holly\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Holly\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Holly\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Holly\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C

error：** NotFoundError: ** Key d not found in checkpoint，我们只想加载`a`，却忽略了变量`d`,  
* 第三种情况：我们想将一个模型的参数加载到另一个模型的计算图中。这也会引发错误，原因：TensorFlow 不知道把加载的所有参数放置到何处。有个  
方法可以提示，就是`var_list`更准确的是`var_list_or_dictionary_mapping_names_to_vars`,这个名字太长了，就用前者  
保存模型时TensorFlow要求使用全局唯一变量的关键原因之一。在保存-模型-文件中，每个保存变量的名称都与其形状和值有关。将其加载到新的计算图中  
与将想要加载的变量的原始名称映射到当前模型的变量中一样

In [1]:
import tensorflow as tf

In [3]:
a = tf.get_variable('a',[])
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.save(sess,'./tftcp.model')

import tensorflow as tf
d = tf.get_variable('d',[])
init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list = {'a':d})
sess = tf.Session()
sess.run(init)
saver.restore(sess,'./tftcp.model')
sess.run(d)

INFO:tensorflow:Restoring parameters from ./tftcp.model


-0.027942419

这是一种关键机制，通过这个机制，可以将没有相同计算图的模型组合在一起。例如，你可能从网上获得了一个训练好的语言模型，希望重用词嵌。或者你  
可能在两次训练之间改变了模型的参数化，想让这个新版本在旧版本的基础上继续前进，但有不想重新训练整个过程，这种情况下，只需手动创建一个字典，  
将旧变量名称映射到新变量即可  
** Note: ** 你要明确的知道正在加载的参数是如何使用的。如果可以，应该使用原作者用来构建的确切代码，以确保计算图的组件与训练时看起来一样  
    如果需要复现模型，务必记住，** 无论多微小的更改，都可能严重损害训练网络的性能，所以始终要将复现结果与原来结果进行对比 **  

### 模型检查

如果想加载的模型来自网络或自己之前创建的，你可能不知道或忘记了原始变量是如何命名的，要检查保存的模型，需要使用TensorFlow库的一些工具  
[tf.contrib.framework.list_variables](https://github.com/tensorflow/tensorflow/blob/r1.11/tensorflow/contrib/framework/python/framework/checkpoint_utils.py)

In [1]:
import tensorflow as tf

a = tf.get_variable('a',[])
b = tf.get_variable('b',[])
c = tf.get_variable('c',[])
init = tf.global_variables_initializer()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
saver.save(sess,'./tftcp.model')
print(tf.contrib.framework.list_variables('./tftcp.model'))

[('a', []), ('b', []), ('c', [])]


In [2]:
?tf.contrib.framework.list_variables()

返回的是一个关于变量的`tuple(name,shape)`这样就可以查找想要的变量名称，同时获得了变量形状

** 这通常会用在使用训练好的模型做迁移学习，我们不会训练所有层的参数，而是固定一些层，通常最开始，我们会固定前面所有层，训练输出层 **